In [ ]:
import numpy as np
import pandas as pd
import os
import gymnasium as gym
from gymnasium import spaces
from scipy.special import softmax
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from typing import Dict, List, Tuple, Optional
from sklearn.preprocessing import MinMaxScaler

class PortfolioEnv(gym.Env):
    metadata = {'render_modes': ['human']}
    
    def __init__(self, stock_data_list: List[pd.DataFrame], episode_length: int = 12, temperature: float = 0.3, window_size: int = 252):
        super(PortfolioEnv, self).__init__()
        
        self.stocks = {f'stock_{i}': df for i, df in enumerate(stock_data_list)}
        self.n_stocks = len(self.stocks)
        self.episode_length = episode_length
        self.temperature = temperature
        self.window_size = window_size  # Window size for feature scaling (252 days = 1 year)
        
        # Use raw features instead of pre-scaled ones
        self.features = [
            'Close', 'MA5', 'MA20', 'MA50', 'MA200',
            'RSI', 'BB_width', 'ATR', 'Return_1W',
            'Return_1M', 'Return_3M', 'CurrentDrawdown',
            'MaxDrawdown_252d', 'Sharpe_20d', 'Sharpe_60d'
        ]
        
        obs_dim = len(self.features) * self.n_stocks
        self.observation_space = spaces.Box(
            low=-1, high=1,
            shape=(obs_dim,), 
            dtype=np.float32
        )
  
        self.action_space = spaces.Box(
            low=-1, high=1,
            shape=(self.n_stocks,),
            dtype=np.float32
        )
        
        self.reset()
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        data_length = len(next(iter(self.stocks.values())))
        max_start_idx = data_length - self.episode_length * 30 - 20
        
        # No minimum start index, as data is assumed to be clean
        self.current_step = np.random.randint(0, max_start_idx)
        self.current_month = 0
        
        self.monthly_returns = []
        self.portfolio_value = 100.0
        self.previous_allocation = np.zeros(self.n_stocks)
        
        observation = self._get_observation()
        info = {}
        
        return observation, info
    
    
    def _get_observation(self):
        observation = []
        for stock_name, stock_data in self.stocks.items():
            # Get window for scaling (including current step)
            window_start = max(0, self.current_step - self.window_size + 1)  # +1 to make room for current step
            window_end = self.current_step + 1  # +1 because slice is exclusive of end index
            window_data = stock_data.iloc[window_start:window_end]
            
            # Scale all features for the entire window
            scaled_features = []
            for feature in self.features:
                scaler = MinMaxScaler(feature_range=(-1, 1))
                feature_values = window_data[feature].values.reshape(-1, 1)
                scaled_window = scaler.fit_transform(feature_values)
                
                # Get the scaled value for the current step (last value in the window)
                scaled_val = scaled_window[-1][0]  # Last row, first column
                scaled_features.append(scaled_val)
            observation.extend(scaled_features)
        return np.array(observation, dtype=np.float32)
    
    # Then update the _convert_to_allocation method:
    def _convert_to_allocation(self, action_weights):
        """
        Improved allocation conversion that handles inputs in the [-1, 1] range
        and converts them to portfolio allocations with constraints.
        """
        # No need to rescale the action_weights from [-1, 1]
        # Softmax works fine with negative values
        
        # Apply softmax with temperature scaling
        raw_allocation = softmax(np.array(action_weights) / self.temperature)
        
        # Calculate initial percent allocations
        percentages = raw_allocation * 100
        
        # Apply discretization constraint (0%, 10%, 20%, 30%)
        # First, find the nearest valid allocation (multiples of 10%)
        allocations = np.round(percentages / 10) * 10
        allocations = np.clip(allocations, 0, 30)
        
        # Determine adjustment needed to sum to 100%
        total_allocation = np.sum(allocations)
        adjustment_needed = 100 - total_allocation
        
        if adjustment_needed != 0:
            # Calculate how close each stock was to the next discretization level
            distance_to_next = np.zeros_like(allocations)
            
            for i in range(len(allocations)):
                if adjustment_needed > 0 and allocations[i] < 30:
                    # If we need to add: how close to rounding up?
                    distance_to_next[i] = 10 - (percentages[i] % 10)
                elif adjustment_needed < 0 and allocations[i] > 0:
                    # If we need to subtract: how close to rounding down?
                    distance_to_next[i] = percentages[i] % 10
                else:
                    # Can't adjust this stock
                    distance_to_next[i] = float('inf')
            
            # Prioritize adjustments for stocks closest to the next level
            num_adjustments = int(abs(adjustment_needed) // 10)
            
            if num_adjustments > 0:
                adjustment_indices = np.argsort(distance_to_next)[:num_adjustments]
                
                for idx in adjustment_indices:
                    if adjustment_needed > 0 and allocations[idx] < 30:
                        allocations[idx] += 10
                        adjustment_needed -= 10
                    elif adjustment_needed < 0 and allocations[idx] > 0:
                        allocations[idx] -= 10
                        adjustment_needed += 10
        
        return allocations
    
    
    def step(self, action):
        allocation = self._convert_to_allocation(action)
        
        self.previous_allocation = allocation.copy()
        
        portfolio_return, stock_returns = self._calculate_monthly_performance(allocation)
        
        self.portfolio_value *= (1 + portfolio_return)
        
        # Get raw metrics (not scaled) for reward calculation
        sharpe = self._calculate_portfolio_metric('Sharpe_20d', allocation)
        max_drawdown = self._calculate_portfolio_metric('MaxDrawdown_252d', allocation)
        
        reward = self._calculate_reward(portfolio_return, sharpe, max_drawdown)
        
        self.monthly_returns.append(portfolio_return)
        
        info = {
            'portfolio_return': portfolio_return,
            'portfolio_value': self.portfolio_value,
            'sharpe': sharpe,
            'max_drawdown': max_drawdown,
            'allocation': allocation.copy(),
            'stock_returns': stock_returns
        }
        
        self.current_step += 30
        self.current_month += 1
        
        terminated = (self.current_month >= self.episode_length)
        truncated = False
        
        observation = self._get_observation()
        
        return observation, reward, terminated, truncated, info
    
    def _calculate_monthly_performance(self, allocation):
        current_prices = np.array([
            self.stocks[f'stock_{i}'].iloc[self.current_step]['Close'] 
            for i in range(self.n_stocks)
        ])
        
        next_step = min(self.current_step + 30, len(next(iter(self.stocks.values()))) - 1)
        next_prices = np.array([
            self.stocks[f'stock_{i}'].iloc[next_step]['Close'] 
            for i in range(self.n_stocks)
        ])
        
        stock_returns = (next_prices - current_prices) / current_prices
        
        portfolio_return = np.sum((allocation / 100) * stock_returns)
        
        return portfolio_return, stock_returns
    
    def _calculate_portfolio_metric(self, metric_name, allocation):
        if not all(metric_name in stock_df.columns for stock_df in self.stocks.values()):
            return 0.0
        
        metric_values = np.array([
            self.stocks[f'stock_{i}'].iloc[self.current_step][metric_name] 
            for i in range(self.n_stocks)
        ])
        return np.sum((allocation / 100) * metric_values)
    
    def _calculate_reward(self, portfolio_return, sharpe, max_drawdown):
        """
        Calculate reward with more balanced risk-return consideration.
        Uses dynamic benchmarking rather than fixed thresholds.
        """
        # Get the average return across all stocks as a benchmark for the current month
        # The Return_1M metric is the return over the past month, not the future month
        benchmark_returns = np.mean([
            self.stocks[f'stock_{i}'].iloc[self.current_step].get('Return_1M', 0)
            for i in range(self.n_stocks)
        ])
        
        # Calculate excess return over benchmark
        excess_return = portfolio_return - max(0, benchmark_returns * 0.01)  # Scaled benchmark
        
        # Base reward from excess return (higher weight for outperformance)
        base_reward = excess_return * 100
        
        # Risk-adjusted components
        sharpe_component = sharpe * 1.0  # Increased weight on Sharpe
        drawdown_component = max_drawdown * -1.5  # Slightly reduced drawdown penalty
        
        # Apply higher penalty for large drawdowns but lower for small ones
        if max_drawdown < -0.1:  # Only penalize significant drawdowns
            drawdown_component *= 1.5
        
        # Combine components
        reward = base_reward + sharpe_component + drawdown_component
        
        return reward
    
    def render(self, mode='human'):
        print(f"Month {self.current_month}")
        print(f"Allocation: {self.previous_allocation}")
        if self.monthly_returns:
            print(f"Last month return: {self.monthly_returns[-1]:.4f}")
            print(f"Portfolio value: {self.portfolio_value:.2f}")
    
    def close(self):
        pass


def train_model(stock_data_list, save_dir='./models', total_timesteps=1_000_000):
    print("Creating environment...")
    env = PortfolioEnv(stock_data_list)
    check_env(env)
    
    print("Initializing PPO agent...")
    model = PPO(
        "MlpPolicy", 
        env,
        verbose=1,
        learning_rate=3e-4,
        gamma=0.99,
        n_steps=2048,
        ent_coef=0.01,
        vf_coef=0.5,
        max_grad_norm=0.5,
        tensorboard_log="./portfolio_env_logs",
        # policy_kwargs={'net_arch': [256, 128, dict(vf=[64], pi=[64])]}
    )
    
    checkpoint_callback = CheckpointCallback(
        save_freq=1000,
        save_path=save_dir,
        name_prefix="ppo_portfolio",
        save_replay_buffer=False,
        save_vecnormalize=False,
    )
    
    print(f"Training for {total_timesteps} timesteps...")
    model.learn(
        total_timesteps=total_timesteps,
        callback=checkpoint_callback,
        progress_bar=True
    )
    
    final_model_path = os.path.join(save_dir, "ppo_portfolio_final")
    model.save(final_model_path)
    print(f"Final model saved to {final_model_path}")
    
    
    return model

def evaluate_model(model, env, n_episodes=10):
    print(f"Evaluating agent over {eval_episodes} episodes...")
    eval_env = Monitor(PortfolioEnv(stock_data_list, temperature=temperature))
    mean_reward, std_reward = evaluate_policy(
        model, 
        eval_env, 
        deterministic=True
    )
    print(f"Mean reward: {mean_reward:.4f} ± {std_reward:.4f}")
    results = detailed_evaluation(model, eval_env, n_episodes=eval_episodes)
    
    print("\nPerformance Summary:")
    print(f"Average Monthly Return: {results['mean_return']:.4f}")
    print(f"Average Sharpe Ratio: {results['mean_sharpe']:.4f}")
    print(f"Average Max Drawdown: {results['mean_drawdown']:.4f}")
    print(f"Final Average Portfolio Value: ${results['mean_final_value']:.2f}")

def detailed_evaluation(model, env, n_episodes=10):
    all_allocations = []
    all_returns = []
    all_sharpes = []
    all_drawdowns = []
    monthly_allocations = []
    final_values = []
    
    for episode in range(n_episodes):
        obs, info = env.reset()
        episode_allocations = []
        episode_returns = []
        episode_sharpes = []
        episode_drawdowns = []
        done = False
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            
            obs, reward, terminated, truncated, info = env.step(action)
            
            done = terminated or truncated
            
            episode_allocations.append(info['allocation'])
            episode_returns.append(info['portfolio_return'])
            episode_sharpes.append(info['sharpe'])
            episode_drawdowns.append(info['max_drawdown'])
            
            if done:
                final_values.append(info['portfolio_value'])
        
        all_allocations.append(episode_allocations)
        monthly_allocations.extend(episode_allocations)
        all_returns.append(np.mean(episode_returns))
        all_sharpes.append(np.mean(episode_sharpes))
        all_drawdowns.append(np.mean(episode_drawdowns))
        
        print(f"Episode {episode+1}: Return = {np.mean(episode_returns):.4f}, Final Value = ${final_values[-1]:.2f}")
    
    avg_allocation = np.mean(monthly_allocations, axis=0)
    
    create_visualizations(
        avg_allocation, 
        all_returns, 
        all_sharpes, 
        all_drawdowns,
        final_values
    )
    
    return {
        'mean_return': np.mean(all_returns),
        'mean_sharpe': np.mean(all_sharpes),
        'mean_drawdown': np.mean(all_drawdowns),
        'mean_final_value': np.mean(final_values),
        'avg_allocation': avg_allocation
    }


def create_visualizations(avg_allocation, returns, sharpes, drawdowns, final_values):
    os.makedirs('results', exist_ok=True)
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(range(len(avg_allocation)), avg_allocation)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 1,
                 f'{height:.1f}%',
                 ha='center', va='bottom', rotation=0)
    
    plt.xlabel('Stock')
    plt.ylabel('Average Allocation (%)')
    plt.title('Average Portfolio Allocation')
    plt.xticks(range(len(avg_allocation)), [f'Stock {i}' for i in range(len(avg_allocation))])
    plt.ylim(0, max(avg_allocation) * 1.2)
    plt.savefig('results/portfolio_allocation.png')
    
    plt.figure(figsize=(10, 6))
    plt.hist(returns, bins=10, alpha=0.7)
    plt.axvline(np.mean(returns), color='r', linestyle='dashed', linewidth=2)
    plt.text(np.mean(returns)*1.1, plt.ylim()[1]*0.9, f'Mean: {np.mean(returns):.4f}')
    plt.xlabel('Average Monthly Return')
    plt.ylabel('Frequency')
    plt.title('Distribution of Average Monthly Returns')
    plt.savefig('results/returns_distribution.png')
    
    plt.figure(figsize=(10, 6))
    plt.hist(final_values, bins=10, alpha=0.7)
    plt.axvline(np.mean(final_values), color='r', linestyle='dashed', linewidth=2)
    plt.text(np.mean(final_values)*1.02, plt.ylim()[1]*0.9, f'Mean: ${np.mean(final_values):.2f}')
    plt.xlabel('Final Portfolio Value ($)')
    plt.ylabel('Frequency')
    plt.title('Distribution of Final Portfolio Values (12-month episodes)')
    plt.savefig('results/portfolio_values.png')
    
    plt.figure(figsize=(12, 6))
    metrics = ['Return (%)', 'Sharpe', 'Drawdown (%)']
    values = [np.mean(returns)*100, np.mean(sharpes), np.mean(drawdowns)*100]
    colors = ['green', 'blue', 'red']
    
    bars = plt.bar(metrics, values, color=colors)
    plt.title('Average Performance Metrics')
    
    for bar in bars:
        height = bar.get_height()
        sign = "+" if height > 0 else ""
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1 if height > 0 else height - 0.6,
                 f'{sign}{height:.2f}',
                 ha='center', va='bottom' if height > 0 else 'top')
    
    plt.savefig('results/performance_metrics.png')
    
    print("Visualizations saved to 'results' directory")

In [ ]:
# data_dir must be appended before the filename
data_dir = "/Users/newuser/Projects/robust_algo_trader/data/gen_synthetic_data/preprocessed_data"

# Load stock data from directory
stock_data_list = []
instrument_list = ["BIT", "CAQD", "CDUV", "CDZ", "CMA", "CQFV", "DEI", "DNW", "DPJE", "EZIG"] 

for instrument in instrument_list:
    file_path = f"{data_dir}/preprocessed_{instrument}.csv"
    df = pd.read_csv(file_path)
    
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date')
    
    stock_data_list.append(df)
    print(f"Loaded {instrument} with {len(df)} data points")

save_dir = './models'
os.makedirs(save_dir, exist_ok=True)


train_model(
    stock_data_list=stock_data_list,
    save_dir=save_dir,
)

print("Training and evaluation complete!")

Loaded BIT with 2321 data points
Loaded CAQD with 2321 data points
Loaded CDUV with 2321 data points
Loaded CDZ with 2321 data points
Loaded CMA with 2321 data points
Loaded CQFV with 2321 data points
Loaded DEI with 2321 data points
Loaded DNW with 2321 data points
Loaded DPJE with 2321 data points
Loaded EZIG with 2321 data points
Creating environment...
Initializing PPO agent...
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Training for 100000000 timesteps...
Logging to ./portfolio_env_logs/PPO_3


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12       |
|    ep_rew_mean     | 36.1     |
| time/              |          |
|    fps             | 73       |
|    iterations      | 1        |
|    time_elapsed    | 27       |
|    total_timesteps | 2048     |
---------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 37.2        |
| time/                   |             |
|    fps                  | 72          |
|    iterations           | 2           |
|    time_elapsed         | 56          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016131267 |
|    clip_fraction        | 0.149       |
|    clip_range           | 0.2         |
|    entropy_loss         | -14.2       |
|    explained_variance   | 0.000957    |
|    learning_rate        | 0.0003      |
|    loss                 | 71.2        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0335     |
|    std                  | 1           |
|    value_loss           | 222         |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 41.5        |
| time/                   |             |
|    fps                  | 71          |
|    iterations           | 3           |
|    time_elapsed         | 85          |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.020520853 |
|    clip_fraction        | 0.202       |
|    clip_range           | 0.2         |
|    entropy_loss         | -14.2       |
|    explained_variance   | 0.00568     |
|    learning_rate        | 0.0003      |
|    loss                 | 72.9        |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.04       |
|    std                  | 1           |
|    value_loss           | 179         |
-----------------------------------------


...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 99.6        |
| time/                   |             |
|    fps                  | 69          |
|    iterations           | 332         |
|    time_elapsed         | 9731        |
|    total_timesteps      | 679936      |
| train/                  |             |
|    approx_kl            | 0.048270162 |
|    clip_fraction        | 0.452       |
|    clip_range           | 0.2         |
|    entropy_loss         | -15.5       |
|    explained_variance   | 0.182       |
|    learning_rate        | 0.0003      |
|    loss                 | 227         |
|    n_updates            | 3310        |
|    policy_gradient_loss | -0.08       |
|    std                  | 1.15        |
|    value_loss           | 597         |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 96.5        |
| time/                   |             |
|    fps                  | 69          |
|    iterations           | 333         |
|    time_elapsed         | 9758        |
|    total_timesteps      | 681984      |
| train/                  |             |
|    approx_kl            | 0.047149524 |
|    clip_fraction        | 0.458       |
|    clip_range           | 0.2         |
|    entropy_loss         | -15.5       |
|    explained_variance   | 0.251       |
|    learning_rate        | 0.0003      |
|    loss                 | 247         |
|    n_updates            | 3320        |
|    policy_gradient_loss | -0.0808     |
|    std                  | 1.15        |
|    value_loss           | 603         |
-----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 12         |
|    ep_rew_mean          | 98         |
| time/                   |            |
|    fps                  | 69         |
|    iterations           | 334        |
|    time_elapsed         | 9784       |
|    total_timesteps      | 684032     |
| train/                  |            |
|    approx_kl            | 0.04495148 |
|    clip_fraction        | 0.43       |
|    clip_range           | 0.2        |
|    entropy_loss         | -15.6      |
|    explained_variance   | 0.226      |
|    learning_rate        | 0.0003     |
|    loss                 | 295        |
|    n_updates            | 3330       |
|    policy_gradient_loss | -0.0776    |
|    std                  | 1.16       |
|    value_loss           | 616        |
----------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 12        |
|    ep_rew_mean          | 99.4      |
| time/                   |           |
|    fps                  | 69        |
|    iterations           | 335       |
|    time_elapsed         | 9811      |
|    total_timesteps      | 686080    |
| train/                  |           |
|    approx_kl            | 0.0455729 |
|    clip_fraction        | 0.428     |
|    clip_range           | 0.2       |
|    entropy_loss         | -15.6     |
|    explained_variance   | 0.212     |
|    learning_rate        | 0.0003    |
|    loss                 | 250       |
|    n_updates            | 3340      |
|    policy_gradient_loss | -0.0763   |
|    std                  | 1.16      |
|    value_loss           | 625       |
---------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 92.5        |
| time/                   |             |
|    fps                  | 69          |
|    iterations           | 336         |
|    time_elapsed         | 9838        |
|    total_timesteps      | 688128      |
| train/                  |             |
|    approx_kl            | 0.045847185 |
|    clip_fraction        | 0.449       |
|    clip_range           | 0.2         |
|    entropy_loss         | -15.6       |
|    explained_variance   | 0.178       |
|    learning_rate        | 0.0003      |
|    loss                 | 319         |
|    n_updates            | 3350        |
|    policy_gradient_loss | -0.0809     |
|    std                  | 1.16        |
|    value_loss           | 631         |
-----------------------------------------
